In [1]:
#Importing libraries
import bempp.api
import numpy as np 
import time
import psutil
import trimesh
from readoff import *
from readpqr import *
start = time.time()

In [2]:
#Main Data to modify.
#Physical parameters.
em = 4.    #[-] Interior electrical permittivity of the solute.
es = 80.   #[-] Exterior electrical permittivity of the solvent.
k = 0.125  #[1/A] Inverse of the Debey-Huckel length of the fluid in the solvent.

#Choice of solute.
Mesh1 ='Mallas_S/Sphere65R4.off'  #Location of the surface mesh of the solute for which the energy is to be calculated. In "off" format
PQR = 'PQR/Sphere5Q3.pqr'        #Location of the position and charges of the solute for which the energy is to be calculated. In "pqr" format

#Choice to solve the matrix equation in BEM/BEM.
#True: Use the Mass Matrix preconditioner.
#False: No preconditioner is used.
SF = False  #Strong form. 
    
#Assembly of border operators.
#fmm: For molecules with a greater number of vertices.
#default_nonlocal: For molecules with a small number of vertices.    
Assemble = 'default_nonlocal' 

#Important parameters of the GMRES.
Tol =1e-6    #GMRES tolerance.
Res =70      #Restart of the GMRES in each iteration.

#Secondary parameters when working with the fmm assembly.
bempp.api.GLOBAL_PARAMETERS.fmm.expansion_order = 5  
bempp.api.GLOBAL_PARAMETERS.fmm.ncrit = 400          
bempp.api.GLOBAL_PARAMETERS.quadrature.regular = 4   

In [3]:
#Data on the position in space, charge and radii of the atoms of the molecule.
PC,Q,R = readpqr(PQR)   

In [4]:
#Generate the surface mesh of the molecule.
V1,F1 = read_off(Mesh1) 
#In case the mesh has small gaps, with trimesh the information of the original mesh without the gaps is obtained.
meshSP = trimesh.Trimesh(vertices = V1, faces= F1) 
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split)) #1 mesh means no cavity.

vertices_split1 = mesh_split[0].vertices 
faces_split1 = mesh_split[0].faces   
grid = bempp.api.grid.grid.Grid(vertices_split1.transpose(), faces_split1.transpose()) #Creation of the surface mesh.

Found 1 meshes


In [5]:
#Generate functional spaces of the potential and its derivative.
dirichl_space = bempp.api.function_space(grid, "P", 1)   #Electrostatic potential at the interface.
neumann_space = bempp.api.function_space(grid, "P", 1)   #Derived from the electrostatic potential at the interface.
print("DS dofs: {0}".format(dirichl_space.global_dof_count))
print("NS dofs: {0}".format(neumann_space.global_dof_count))  

DS dofs: 7908
NS dofs: 7908


In [6]:
#Generate the boundary operators.
#Domain of the solute Ωm.
IL = bempp.api.operators.boundary.sparse.identity(dirichl_space, dirichl_space, dirichl_space) #1
KL = bempp.api.operators.boundary.laplace.double_layer(dirichl_space, dirichl_space, dirichl_space, assembler=Assemble) #K
VL = bempp.api.operators.boundary.laplace.single_layer(neumann_space, dirichl_space, dirichl_space, assembler=Assemble) #V
#Domain of the solvent Ωs.
IH = bempp.api.operators.boundary.sparse.identity(dirichl_space, neumann_space, neumann_space) #1
if k==0:
    KH = bempp.api.operators.boundary.laplace.double_layer(dirichl_space, neumann_space, neumann_space, assembler=Assemble) #K
    VH = bempp.api.operators.boundary.laplace.single_layer(neumann_space, neumann_space, neumann_space, assembler=Assemble) #V
else:
    KH = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space, neumann_space, neumann_space, k, assembler=Assemble) #K
    VH = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space, neumann_space, neumann_space, k, assembler=Assemble) #V

In [7]:
#Creation of the Coulomb potential function.  
@bempp.api.complex_callable(jit=False)
def U_c(x, n, domain_index, result):
    global Q,PC,em
    result[:] = (1 / (4.*np.pi*em))  * np.sum( Q / np.linalg.norm( x - PC, axis=1))
U_c = bempp.api.GridFunction(dirichl_space, fun=U_c)

#Construction of the right vector.
if SF==False:
    # Rhs in Ωm.
    rhs_M = (U_c).projections(dirichl_space)
    # Rhs in Ωs.
    rhs_S = np.zeros(neumann_space.global_dof_count)
    # The combination of Rhs.
    rhs = np.concatenate([rhs_M, rhs_S])
else:    
    rhs = [IL*U_c, 0*IH*U_c]

In [8]:
#Construction left 2x2 matrix.
if SF==False:
    #Position of the 2x2 matrix.
    blocks = [[None,None],[None,None]] 
    blocks[0][0] = (0.5*IL + KL).weak_form()  #0.5+K
    blocks[0][1] = -VL.weak_form()            #-V
    blocks[1][0] = (0.5*IH - KH).weak_form()  #0.5-K
    blocks[1][1] = (em/es)*VH.weak_form()     #V(em/es)    
    blocked = bempp.api.assembly.blocked_operator.BlockedDiscreteOperator(np.array(blocks))   
    #Block diagonal preconditioner for BEM.
    from preconditioners import *
    P = BlockDiagonal_2x2(dirichl_space, neumann_space, blocks, es, em, k) 
else:   
    #Position of the 2x2 matrix.
    blocks = bempp.api.BlockedOperator(2,2)    
    blocks[0,0] = (0.5*IL + KL)  #0.5+K
    blocks[0,1] = -VL            #-V
    blocks[1,0] = (0.5*IH - KH)  #0.5-K
    blocks[1,1] = VH*(em/es)     #V(em/es)

In [9]:
#The solution of the matrix equation Ax=B is solved.
#Iteration counter.
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 100) == (it_count // 100):
        print(it_count,x)

# Solution by GMRES.
from scipy.sparse.linalg import gmres
if SF==False:
    start1 = time.time()
    soln, info = gmres(blocked, rhs, M=P, callback=count_iterations,tol=Tol, restart=Res)  
    end1 = time.time() 
else:
    start1 = time.time()
    soln, info, res, it_count = bempp.api.linalg.gmres(blocks, rhs, return_residuals=True, return_iteration_count=True, use_strong_form=True, tol=Tol, restart=Res)
    end1 = time.time()
    
# Time to solve the equation.
curr_time1 = (end1 - start1)
print("Number of GMRES iterations: {0}".format(it_count))
print("Total time in GMRES: {:5.2f} [s]".format(curr_time1))

Number of GMRES iterations: 28
Total time in GMRES: 14.47 [s]


In [10]:
#Calculates the entire global domain of the potential from the solution of the calculated edge.
if SF==False:
    # Solution for Dirichlet data.
    soln_u = soln[:dirichl_space.global_dof_count]
    dirichlet_fun = bempp.api.GridFunction(dirichl_space, coefficients=soln_u)    
    # Solution for Neumann data.
    soln_du = soln[dirichl_space.global_dof_count:] 
    neumann_fun = bempp.api.GridFunction(neumann_space, coefficients=soln_du)
else:
    dirichlet_fun = soln[0]
    neumann_fun = soln[1]

In [11]:
#Calculation of the potential in the position of the atoms of the molecule.
VF = bempp.api.operators.potential.laplace.single_layer(neumann_space, np.transpose(PC)) 
KF = bempp.api.operators.potential.laplace.double_layer(dirichl_space, np.transpose(PC))
uF = VF*neumann_fun - KF*dirichlet_fun

#Result of the total solvation energy.
E_Solv = 0.5*4.*np.pi*332.064*np.sum(Q*uF).real 
print('Solvation Energy: {:7.6f} [kCal/mol]'.format(E_Solv) )

#Total time.
end = time.time()
curr_time = (end - start)
print("Total time: {:5.2f} [s]".format(curr_time))

Solvation Energy: -55.901478 [kCal/mol]
Total time: 184.56 [s]
